In [1]:
import os
import sys
import pandas as pd

sys.path.append('/home/fboehning/fboehning')
#sys.path.append('C:/Users/felix/Documents/Studium Ingenieurinformatik/dAISY/fboehning')

import logging
logging.basicConfig(
    level=logging.INFO,
    #level=logging.DEBUG,
    format='%(asctime)s - %(filename)s - %(name)s - %(funcName)s - %(levelname)s - %(message)s')

In [2]:
TOOLKIT=[
    'beesup_llm.toolkit.llm_utils',
    'beesup_llm.toolkit.dict_utils',
    'beesup_llm.toolkit.setup_utils',
    'beesup_llm.toolkit.visualization',
    'beesup_llm.toolkit.display_settings',
]

DATASET=[
    'beesup_llm.dataset',
]

MODEL_PIPELINES=[
    'beesup_llm.model_pipelines'
]

INJECTION=[
    'beesup_llm.injection',
    'beesup_llm.injection.taxomizer',
    'beesup_llm.injection.taxomizer_utils',
    #'beesup_llm.injection.evaluation_pipeline',
    #'beesup_llm.injection.evaluation_utils'
]

EXPERIMENT=[
    'beesup_llm.experiment',
    'beesup_llm.experiment.injection_experiment'
]

import importlib
import beesup_llm;
import beesup_llm.toolkit.llm_utils; from beesup_llm.toolkit.llm_utils import *
import beesup_llm.toolkit.dict_utils; from beesup_llm.toolkit.dict_utils import *
import beesup_llm.toolkit.setup_utils; from beesup_llm.toolkit.setup_utils import *
import beesup_llm.toolkit.visualization; from beesup_llm.toolkit.visualization import *
import beesup_llm.toolkit.display_settings; from beesup_llm.toolkit.display_settings import *

import beesup_llm.dataset; from beesup_llm.dataset import *
import beesup_llm.training; from beesup_llm.training import *
import beesup_llm.model_pipelines; from beesup_llm.model_pipelines import *

# import beesup_llm.extraction; from beesup_llm.extraction import *
# import beesup_llm.extraction.extraction_utils; from beesup_llm.extraction.extraction_utils import *
# import beesup_llm.extraction.extraction_pipeline; from beesup_llm.extraction.extraction_pipeline import *

# import beesup_llm.extraction.evaluation_utils; from beesup_llm.extraction.evaluation_utils import *
# import beesup_llm.extraction.evaluation_pipeline; from beesup_llm.extraction.evaluation_pipeline import *

# import beesup_llm.experiment; from beesup_llm.experiment import *
# import beesup_llm.experiment.extraction_experiment; from beesup_llm.experiment.extraction_experiment import *

import beesup_llm.experiment; from beesup_llm.experiment import *
import beesup_llm.experiment.injection_experiment; from beesup_llm.experiment.injection_experiment import *

import beesup_llm.injection; from beesup_llm.injection import *
import beesup_llm.injection.taxomizer_utils; from beesup_llm.injection.taxomizer_utils import *
import beesup_llm.injection.taxomizer; from beesup_llm.injection.taxomizer import *

def reimport(pathlist=TOOLKIT+DATASET+MODEL_PIPELINES+INJECTION+EXPERIMENT):

    pathlist = ['beesup_llm'] + pathlist

    for module_path in pathlist:
        if module_path in sys.modules:
            # Reload if the module is already imported
            importlib.reload(sys.modules[module_path])
        else:

            importlib.import_module(module_path)
            # Import the module or submodule for the first time
            globals()[module_path.split('.')[0]] = importlib.import_module(module_path)

        module = sys.modules[module_path]
        globals().update({name: getattr(module, name) for name in dir(module) if not name.startswith('_')})

import logging
def set_info():
    logger = logging.getLogger('beesup_llm')
    logger.setLevel(logging.INFO)
    logging.getLogger().setLevel(logging.INFO)

def set_debug():
    logger = logging.getLogger('beesup_llm')
    logger.setLevel(logging.DEBUG)
    logging.getLogger().setLevel(logging.DEBUG)

set_info()
reimport()

2025-02-07 08:23:20,617 - config.py - datasets - <module> - INFO - PyTorch version 2.3.1 available.


In [3]:
#Load Model
llm_pipe=LlamaPipeline(1)
model=llm_pipe.get_model()

2025-02-07 08:23:25,422 - __init__.py - beesup_llm - __init__ - INFO - 0001_LLM_PIPELINE initialised
2025-02-07 08:23:25,425 - __init__.py - beesup_llm - load_model - INFO - Loading model meta-llama/Meta-Llama-3.1-8B-Instruct
2025-02-07 08:23:26,339 - modeling.py - accelerate.utils.modeling - get_balanced_memory - INFO - We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
llm_pipe=LlamaPipeline(1,model=model)
llm_pipe.prepare_inference()

2025-02-07 08:24:22,417 - __init__.py - beesup_llm - __init__ - INFO - 0001_LLM_PIPELINE initialised


# Generate Taxonomy

In [7]:
reimport()

In [30]:
BaseDataset.get_overview(keypaths=['emb_model_config.task','remarks','emb_model_config.name_or_path'])

,id,name,remarks,task,name_or_path
0,1,0001_dataset,"basic knowledge_df from wildbienen_db, no embe...",NaN,NaN
1,2,0002_dataset,"knowledge dataset with embeddings in subject, ...",separation,jinaai/jina-embeddings-v3
2,3,0003_dataset,"knowledge dataset with embeddings in subject, ...",classification,jinaai/jina-embeddings-v3
3,4,0004_dataset,"knowledge dataset with embeddings in subject, ...",retrieval.passage,jinaai/jina-embeddings-v3
4,5,0005_dataset,"knowledge dataset with embeddings in subject, ...",text-matching,jinaai/jina-embeddings-v3
5,6,0006_dataset,"knowledge dataset with embeddings in subject, ...",NaN,nvidia/NV-Embed-v2


In [9]:
knowledge=BaseDataset(6)
knowledge_df=knowledge.df
knowledge_df=knowledge_df[knowledge_df.p!='Wissenschaftlicher Name'].copy()

def get_first_n_words(text, n=2):
    return ' '.join(re.split(r'[\s,;]',text)[:n])

knowledge_df['label']=knowledge_df.apply(lambda x: f'{get_first_n_words(x.s)}/ {x.p}', axis=1)

def compose_emb(row):
    s,p,o,so,po,spo=[row[f'{c}_emb'] for c in ['s','p','o','so','po','spo']]

    if row.attr_type=='Text':
        emb=po
    elif row.attr_type=='Attribute':
        emb=po

    # weights = np.array([0.0, 0.25, 0.75]) 
    # emb = np.average(np.array([s,p,o]), axis=0, weights=weights)
    #emb=spo-s
    #emb=po

    emb_normalized=emb/ np.linalg.norm(emb) #normalize
    return emb_normalized

chunks_df=knowledge_df.copy()
chunks_df['emb']=knowledge_df.apply(compose_emb, axis=1)
chunks_df=chunks_df[['spo','s','p','o','source_name','attr_type','n_units','n_words','label','emb']].copy()
chunks_df['label']=chunks_df.apply(lambda x: f"{get_first_n_words(x.p, n=4)}/ {get_first_n_words(x.o, n=10)}", axis=1)

chunks_df.rename(columns={
    'spo':'chunk',
    's':'subject',
    'p':'predicate',
    'o':'object',
    },inplace=True)

DROP_EMB_DUPLICATES=False
if DROP_EMB_DUPLICATES:
    chunks_df['emb_tuple']=chunks_df['emb'].apply(tuple)
    grouped = chunks_df.groupby('emb_tuple', group_keys=False).apply(lambda group: list(group.index), include_groups=False).reset_index(name='kidc')
    chunks_df = pd.merge(chunks_df, grouped, on='emb_tuple', how='left')
    chunks_df.drop_duplicates(subset='emb_tuple',keep='first',inplace=True)
    chunks_df.drop(columns=['emb_tuple'],inplace=True)
else:
    chunks_df = chunks_df.reset_index(names='kidx')

chunks_df

2025-02-07 08:31:06,562 - __init__.py - beesup_llm - __init__ - INFO - 0006_DATASET initialised


,kidx,chunk,subject,predicate,object,source_name,attr_type,n_units,n_words,label,emb
0,0,"Aglaoapis tridentata (Nylander, 1848)\nAussehe...","Aglaoapis tridentata (Nylander, 1848)",Aussehen,"Die Biene ist schwarz, etwa 8–10 mm lang und m...",wikipedia,Text,1,33,Aussehen/ Die Biene ist schwarz etwa 8–10 mm ...,"[0.013315118, -0.0025690868, -0.002520601, -0...."
1,1,"Aglaoapis tridentata (Nylander, 1848)\nFamilie...","Aglaoapis tridentata (Nylander, 1848)",Familie,Megachilidae,wildbienen-kataster,Attribute,1,1,Familie/ Megachilidae,"[-0.008217988, -0.017626682, 0.014735313, 0.00..."
2,2,"Aglaoapis tridentata (Nylander, 1848)\nKurzbes...","Aglaoapis tridentata (Nylander, 1848)",Kurzbeschreibung,"Dunkle Zweizahnbiene (Aglaoapis tridentata, Sy...",wikipedia,Text,1,23,Kurzbeschreibung/ Dunkle Zweizahnbiene (Aglaoa...,"[0.0050212676, -0.020270795, 0.03047962, -0.01..."
3,3,"Aglaoapis tridentata (Nylander, 1848)\nLebensr...","Aglaoapis tridentata (Nylander, 1848)",Lebensraum,"Die Biene lebt auf trockenen, blütenreichen Bö...",wikipedia,Text,1,38,Lebensraum/ Die Biene lebt auf trockenen blüt...,"[0.010719752, 0.007365582, 0.02396487, -0.0069..."
4,4,"Aglaoapis tridentata (Nylander, 1848)\nSozialv...","Aglaoapis tridentata (Nylander, 1848)",Sozialverhalten,parasitisch,wildbienen-kataster,Attribute,1,1,Sozialverhalten/ parasitisch,"[0.0032968458, 0.008870961, 0.024923155, 0.009..."
...,...,...,...,...,...,...,...,...,...,...,...
3407,3883,"Xylocopa violacea (Linnaeus, 1758)\nNistweise\...","Xylocopa violacea (Linnaeus, 1758)",Nistweise,hypergäisch-selbstgegrabene Hohlräume,wildbienen-kataster,Attribute,1,2,Nistweise/ hypergäisch-selbstgegrabene Hohlräume,"[0.0011781659, 0.0005463787, 0.036551803, 0.00..."
3408,3884,"Xylocopa violacea (Linnaeus, 1758)\nPollensamm...","Xylocopa violacea (Linnaeus, 1758)",Pollensammelverhalten,polylektisch,wildbienen-kataster,Attribute,1,1,Pollensammelverhalten/ polylektisch,"[0.0017042567, -0.013260176, 0.028913721, 0.00..."
3409,3885,"Xylocopa violacea (Linnaeus, 1758)\nRequisiten...","Xylocopa violacea (Linnaeus, 1758)",Requisiten,Totholz,wildbienen-kataster,Attribute,1,1,Requisiten/ Totholz,"[0.00826123, -0.0013367897, 0.031230105, -0.00..."
3410,3886,"Xylocopa violacea (Linnaeus, 1758)\nUmgangsspr...","Xylocopa violacea (Linnaeus, 1758)",Umgangssprachliche Bezeichnungen,Blauschwarze Holzbiene; Große Holzbiene,wildbienen-kataster,Attribute,2,4,Umgangssprachliche Bezeichnungen/ Blauschwarze...,"[0.02123162, 0.0107286265, 0.011301344, -0.010..."


In [13]:
taxomizer=Taxomizer(
    dist_flattening_config=dict(use_kneepoint=True),
    ddist_flattening_config=dict(use_kneepoint=True),
    llm_ref=llm_pipe
    )
taxomizer.get_config()

2025-02-07 08:32:56,441 - __init__.py - beesup_llm - __init__ - INFO - 0002_TAXOMIZER initialised
2025-02-07 08:32:56,453 - __init__.py - beesup_llm - __init__ - INFO - 0001_LLM_PIPELINE initialised


{'type': 'taxomizer',
 'id': 2,
 'name': '0002_taxomizer',
 'dir_name': 'taxomizers',
 'lab_name': 'injection_lab',
 'rel_path': 'injection_lab/taxomizers/0002_taxomizer',
 'dist_flattening_config': {'include_leaves': False,
  'use_kneepoint': True,
  'use_std': False,
  'std_factor': 1.0},
 'ddist_flattening_config': {'include_leaves': False,
  'use_kneepoint': True,
  'use_std': False,
  'std_factor': 1.0},
 'linkage_args': {'method': 'ward', 'optimal_ordering': False},
 'llm_config': {'type': 'llm_pipeline',
  'id': 1,
  'name': '0001_llm_pipeline',
  'dir_name': 'llm_pipelines',
  'lab_name': 'injection_lab',
  'rel_path': 'injection_lab/llm_pipelines/0001_llm_pipeline',
  'name_or_path': 'meta-llama/Meta-Llama-3.1-8B-Instruct',
  'pipeline_args': {'return_full_text': False,
   'clean_up_tokenization_spaces': True},
  'bnb_config': {'load_in_4bit': True,
   'bnb_4bit_use_double_quant': True,
   'bnb_4bit_quant_type': 'nf4'},
  'inference_tokenizer_config': {'padding_side': 'left',


In [15]:
taxomizer.spawn()

2025-02-07 08:34:04,734 - __init__.py - beesup_llm - spawn_config - INFO - 0002_TAXOMIZER config spawned at injection_lab/taxomizers/0002_taxomizer


In [16]:
taxomizer.process(chunks_df, verbose=True)

2025-02-07 08:34:12,837 - taxomizer_utils.py - root - linkage_to_btree - INFO - {'num_nodes': 6823, 'height': 127, 'num_root_children': 2, 'num_leaves': 3412, 'branching': 2.0, 'avg_degree': 1.0, 'avg_balance': 4.32, 'dia': 236, 'avg_depth': 19.63, 'width': 656}
2025-02-07 08:34:13,012 - taxomizer_utils.py - root - get_dist_kneepoint - INFO - knee_dist: 1.0658, knee_index: 3369/ 3411
2025-02-07 08:34:13,369 - taxomizer_utils.py - root - do_dist_flattening - INFO - {'num_nodes': 3454, 'height': 9, 'num_root_children': 2, 'num_leaves': 3412, 'branching': 84.17, 'avg_degree': 1.0, 'avg_balance': 0.01, 'dia': 16, 'avg_depth': 5.24, 'width': 897}
2025-02-07 08:34:13,411 - taxomizer_utils.py - root - get_ddist_kneepoint - INFO - knee_ddist: 3.4228, knee_index: 35/ 41
2025-02-07 08:34:13,925 - taxomizer_utils.py - root - do_ddist_flattening - INFO - {'num_nodes': 3427, 'height': 4, 'num_root_children': 2, 'num_leaves': 3412, 'branching': 244.57, 'avg_degree': 1.0, 'avg_balance': 0.0, 'dia': 7

3412, 3413, 1/17                                        
2/17                                                                       
3/17                                                                       
4/17                                                                       
5/17                                                                       
6/17                                                                       
7/17                                                                       
8/17                                                                       
9/17                                                                       
10/17                                                                    
11/17                                                                    
12/17                                                                     
13/17                                                                    
14/17                                 

2025-02-07 08:35:34,566 - taxomizer.py - beesup_llm - get_embedding_tree - INFO - all chunks included in the tree: True


[6822] Wildbienen7971622943878174                                        
├──  [6820] Nist- und Bauverhalten von Wildbienen
│   ├──  [6796] Familienübersicht der Wildbienen
│   ├──  [6799] Pollensammelverhalten: Vielfalt und Präferenzen
│   ├──  [6806] Nistplatzwahl und Nistplatztypen
│   ├──  [6793] Sozialverhalten von Wildbienen
│   └──  [6807] Baumaterialien und Requisiten für die Nist- und Bauweise von Wildbienen
└──  [6821] Umgangssprachliche Bezeichnungen der Wildbienen
    ├──  [6817] Lebensweise und Merkmale
    │   ├──  [3412] Lebensweise und Nestbau


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


    │   ├──  [6798] Merkmale der Wildbienen
    │   ├──  [6797] Vorkommen und Lebensweise
    │   └──  [6801] Verbreitung und Lebensraum
    └──  [6818] Vorkommen und Lokalitäten der Dünen-Steppenbiene
        ├──  [3413] Umgangssprachliche Bezeichnungen der Wildbienen-Arten
        ├──  [6810] Kuckuckswespen (Nomaden)
        └──  [6812] Wildbienenarten der Dünen-Steppenbiene


In [7]:
taxomizer=Taxomizer(2)
taxomizer.get_config()

2025-02-07 08:18:41,910 - __init__.py - beesup_llm - __init__ - INFO - 0001_TAXOMIZER initialised


{'type': 'taxomizer',
 'id': 1,
 'name': '0001_taxomizer',
 'dir_name': 'taxomizers',
 'lab_name': 'injection_lab',
 'rel_path': 'injection_lab/taxomizers/0001_taxomizer',
 'dist_flattening_config': {'include_leaves': False,
  'use_kneepoint': True,
  'use_std': False,
  'std_factor': 1.0},
 'ddist_flattening_config': {'include_leaves': False,
  'use_kneepoint': True,
  'use_std': False,
  'std_factor': 1.0},
 'linkage_args': {'method': 'ward', 'optimal_ordering': False},
 'llm_config': {'type': 'llm_pipeline',
  'id': 2,
  'name': '0002_llm_pipeline',
  'dir_name': 'llm_pipelines',
  'lab_name': 'injection_lab',
  'rel_path': 'injection_lab/llm_pipelines/0002_llm_pipeline',
  'name_or_path': 'meta-llama/Meta-Llama-3.1-8B-Instruct',
  'pipeline_args': {'return_full_text': False,
   'clean_up_tokenization_spaces': True},
  'bnb_config': {'load_in_4bit': True,
   'bnb_4bit_use_double_quant': True,
   'bnb_4bit_quant_type': 'nf4'},
  'inference_tokenizer_config': {'padding_side': 'left',


# Create Instruction Finetuning Dataset

- also consider finetuning not in chat-format
- provide whole table of content in system instruction?
- multiple queries in one conversation (alternating masking required)
- consider entitie-swap.. genus species (author, year) --> german synonym 
- include source in the query prompt .. "What does wikipedia write about _predicate_ of _subject_?"

In [44]:
taxomizer=Taxomizer(2)
taxomizer.get_config()

2025-02-07 09:05:23,949 - __init__.py - beesup_llm - __init__ - INFO - 0002_TAXOMIZER initialised


{'type': 'taxomizer',
 'id': 2,
 'name': '0002_taxomizer',
 'dir_name': 'taxomizers',
 'lab_name': 'injection_lab',
 'rel_path': 'injection_lab/taxomizers/0002_taxomizer',
 'dist_flattening_config': {'include_leaves': False,
  'use_kneepoint': True,
  'use_std': False,
  'std_factor': 1.0},
 'ddist_flattening_config': {'include_leaves': False,
  'use_kneepoint': True,
  'use_std': False,
  'std_factor': 1.0},
 'linkage_args': {'method': 'ward', 'optimal_ordering': False},
 'llm_config': {'type': 'llm_pipeline',
  'id': 1,
  'name': '0001_llm_pipeline',
  'dir_name': 'llm_pipelines',
  'lab_name': 'injection_lab',
  'rel_path': 'injection_lab/llm_pipelines/0001_llm_pipeline',
  'name_or_path': 'meta-llama/Meta-Llama-3.1-8B-Instruct',
  'pipeline_args': {'return_full_text': False,
   'clean_up_tokenization_spaces': True},
  'bnb_config': {'load_in_4bit': True,
   'bnb_4bit_use_double_quant': True,
   'bnb_4bit_quant_type': 'nf4'},
  'inference_tokenizer_config': {'padding_side': 'left',


In [31]:
taxomizer=Taxomizer(2)
taxomizer.get_config()
tree=taxomizer.header_tree
taxomizer_df=taxomizer.df.copy()
taxomizer_df.sort_values(by='node_id',ascending=False,inplace=True)
taxomizer_df.head(3)

2025-02-07 08:52:10,497 - __init__.py - beesup_llm - __init__ - INFO - 0002_TAXOMIZER initialised


,kidx,chunk,subject,predicate,object,source_name,attr_type,n_units,n_words,label,emb,node_id
1692,1916,"Hylaeus communis (Nylander, 1852)\nParasiten\n...","Hylaeus communis (Nylander, 1852)",Parasiten,"""Als Nestparasit wurden Schmalbauchwespen der ...",wikipedia,Text,1,22,"Parasiten/ ""Als Nestparasit wurden Schmalbauch...","[0.010434585, -0.02319299, -0.0005728865, 0.02...",6812
458,527,"Andrena nigroaenea (Kirby, 1802)\nSystematik\n...","Andrena nigroaenea (Kirby, 1802)",Systematik,Von der Art wurden mehrere Unterarten beschrie...,wikipedia,Text,1,27,Systematik/ Von der Art wurden mehrere Unterar...,"[0.013902873, -0.018704087, 0.012390775, -0.00...",6812
939,1070,"Bombus distinguendus (Morawitz, 1869)\nKurzbes...","Bombus distinguendus (Morawitz, 1869)",Kurzbeschreibung,Die Deichhummel (Bombus distinguendus) ist ein...,wikipedia,Text,1,10,Kurzbeschreibung/ Die Deichhummel (Bombus dist...,"[0.004863024, 0.006708058, 0.0067056306, 0.007...",6812


In [32]:
# INSTRUCTION PROMPTING
def get_n_words_str(n_words, lang='de'):
    n_words_str=None

    if lang=='de':
        if n_words==1: n_words_str='einem Wort'
        elif n_words==2: n_words_str='zwei Wörtern'
        elif n_words==3: n_words_str='drei Wörtern'
        elif n_words==4: n_words_str='vier Wörtern'
        elif n_words==5: n_words_str='fünf Wörtern'
        elif n_words < 10: n_words_str='weniger als zehn Wörtern'
        elif n_words < 20: n_words_str='weniger als zwanzig Wörtern'
        else:
            n_words_str=f'etwa {round(n_words, -1)} Wörtern'
    
    if lang =='en':
        if n_words==1: n_words_str='one word'
        elif n_words==2: n_words_str='two words'
        elif n_words==3: n_words_str='three words'
        elif n_words==4: n_words_str='four words'
        elif n_words==5: n_words_str='five words'
        elif n_words < 10: n_words_str='less than ten words'
        elif n_words < 20: n_words_str='less than twenty words'
        else:
            n_words_str=f'approximately {round(n_words, -1)} words'
    
    return n_words_str

def get_system_prompt():
    prompt=""
    prompt+="""
You are a knowledge-oriented AI assistant. \
You provide factual knowledge about the user's input in the style of a lexicon entry, without any additional commentary, conversational language, or speculation. \
You only respond in German, using single words or brief descriptions like in an encyclopedia. \
""".strip()
    return prompt
    
def get_query_prompt(sample):

    prompt=""
    #prompt+=f"Innerhalb eines Wissensbaumes befinden wir uns an folgender Stelle:\n"
    prompt+="{toc}\n\n".format(**sample)

    prompt+="Was weißt du über die Eigenschaft '{predicate}' von {subject}? ".format(**sample)
    prompt+=f"Antworte mit {get_n_words_str(sample.n_words, lang='de')}!"

    return prompt

def get_prompt_messages(sample):
    
    prompt_messages=[]
    prompt_messages.append({'role':'system','content':get_system_prompt()})
    prompt_messages.append({'role':'user','content':get_query_prompt(sample)})

    return prompt_messages


In [33]:
# Create Finetuning dataset
dataset_df=taxomizer_df.copy()

toc_dict=dict()
hpath_dict=dict()
for node_id in dataset_df.node_id.unique():
    the_node=get_node(node_id, tree)
    hpath_dict[node_id]=get_headerpath_until(the_node,tree)
    toc_dict[node_id]=get_table_of_contents(get_tree_branch_until(the_node, tree))

dataset_df['toc']=dataset_df.node_id.map(toc_dict)
dataset_df['hpath']=dataset_df.node_id.map(hpath_dict)

dataset_df['prompt_messages']=dataset_df.apply(lambda x: get_prompt_messages(x), axis=1)
dataset_df['gold_message']=dataset_df['object'].apply(lambda x: [{'role':'assistant','content':x}])
dataset_df.reset_index(drop=False, names='tidx', inplace=True) #preserve taxomizer index as tidx

# add embedding combinations
emb_cols=[f"{c}_emb" for c in ['s','p','o','so','po','spo']]
dataset_df=dataset_df.merge(knowledge_df[emb_cols], left_on='kidx', right_index=True, how='left')
dataset_df

,tidx,kidx,chunk,subject,predicate,object,source_name,attr_type,n_units,n_words,...,toc,hpath,prompt_messages,gold_message,s_emb,p_emb,o_emb,so_emb,po_emb,spo_emb
0,1692,1916,"Hylaeus communis (Nylander, 1852)\nParasiten\n...","Hylaeus communis (Nylander, 1852)",Parasiten,"""Als Nestparasit wurden Schmalbauchwespen der ...",wikipedia,Text,1,22,...,Wildbienen\n└── Umgangssprachliche Bezeichnu...,"[Wildbienen, Umgangssprachliche Bezeichnungen ...","[{'role': 'system', 'content': 'You are a know...","[{'role': 'assistant', 'content': '""Als Nestpa...","[0.0074964184, -0.006532411, 0.010707644, 0.00...","[0.0067741987, -0.00579156, 0.012198386, 0.006...","[0.006624135, -0.016266031, 0.0042751855, 0.02...","[0.0175043, -0.012922153, -0.0012822307, 0.013...","[0.010434584, -0.023192989, -0.00057288643, 0....","[0.011627321, -0.015591463, -0.001989269, 0.01..."
1,458,527,"Andrena nigroaenea (Kirby, 1802)\nSystematik\n...","Andrena nigroaenea (Kirby, 1802)",Systematik,Von der Art wurden mehrere Unterarten beschrie...,wikipedia,Text,1,27,...,Wildbienen\n└── Umgangssprachliche Bezeichnu...,"[Wildbienen, Umgangssprachliche Bezeichnungen ...","[{'role': 'system', 'content': 'You are a know...","[{'role': 'assistant', 'content': 'Von der Art...","[-0.003798669, -0.025593871, 0.009554575, -0.0...","[0.008728638, -0.0020928406, 0.026165051, -0.0...","[0.003310302, -0.018220898, 0.018383589, -0.00...","[-0.0030643565, -0.026073407, 0.01752103, -0.0...","[0.013902873, -0.018704087, 0.012390775, -0.00...","[-0.00032512893, -0.028085468, 0.017131323, -0..."
2,939,1070,"Bombus distinguendus (Morawitz, 1869)\nKurzbes...","Bombus distinguendus (Morawitz, 1869)",Kurzbeschreibung,Die Deichhummel (Bombus distinguendus) ist ein...,wikipedia,Text,1,10,...,Wildbienen\n└── Umgangssprachliche Bezeichnu...,"[Wildbienen, Umgangssprachliche Bezeichnungen ...","[{'role': 'system', 'content': 'You are a know...","[{'role': 'assistant', 'content': 'Die Deichhu...","[0.0035069948, -0.030443307, 0.00828194, 0.002...","[0.005835669, 0.00046010816, 0.013336124, 0.00...","[0.0011134875, 0.001320186, 0.0057929945, 0.00...","[0.00798141, -0.002544037, 0.010852625, 0.0044...","[0.0048630238, 0.0067080576, 0.00670563, 0.007...","[0.0026471736, -0.0005944702, 0.012585272, 0.0..."
3,3182,3620,"Sphecodes ephippius (Linnaeus, 1767)\nKurzbesc...","Sphecodes ephippius (Linnaeus, 1767)",Kurzbeschreibung,Sphecodes ephippius ist eine parasitische Bien...,wikipedia,Text,1,20,...,Wildbienen\n└── Umgangssprachliche Bezeichnu...,"[Wildbienen, Umgangssprachliche Bezeichnungen ...","[{'role': 'system', 'content': 'You are a know...","[{'role': 'assistant', 'content': 'Sphecodes e...","[-0.004589162, -0.027405059, -0.0027844561, 0....","[0.005835669, 0.00046010816, 0.013336124, 0.00...","[-0.010704446, 0.006089516, 0.0037873902, 0.00...","[-0.010260817, 0.003773774, 0.007520293, -0.00...","[-0.010285014, 0.010988846, 0.0028065783, 0.00...","[-0.01181556, 0.0044817245, 0.0042099343, 0.00..."
4,1926,2189,"Lasioglossum costulatum (Kriechbaumer, 1873)\n...","Lasioglossum costulatum (Kriechbaumer, 1873)",Kurzbeschreibung,Die Glockenblumen-Schmalbiene (Lasioglossum co...,wikipedia,Text,1,12,...,Wildbienen\n└── Umgangssprachliche Bezeichnu...,"[Wildbienen, Umgangssprachliche Bezeichnungen ...","[{'role': 'system', 'content': 'You are a know...","[{'role': 'assistant', 'content': 'Die Glocken...","[0.012257211, -0.014003468, 0.012141728, 0.003...","[0.005835669, 0.00046010816, 0.013336124, 0.00...","[0.0023955784, -0.0008903179, 0.013634946, 0.0...","[0.0080582155, 0.0026082508, 0.017733106, -0.0...","[0.0026733035, 0.0025412277, 0.010954422, 0.00...","[0.007582612, 0.005349722, 0.015507144, -3.243..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3407,3219,3661,"Sphecodes hyalinatus (Hagens, 1882)\nLebenswei...","Sphecodes hyalinatus (Hagens, 1882)",Lebensweise,Sphecodes ephippus ist wie alle Arten der Gatt...,wikipedia,Text,1,50,...,Wildbienen\n└── Um

In [40]:
taxomizer.get_config()

{'type': 'taxomizer',
 'id': 2,
 'name': '0002_taxomizer',
 'dir_name': 'taxomizers',
 'lab_name': 'injection_lab',
 'rel_path': 'injection_lab/taxomizers/0002_taxomizer',
 'dist_flattening_config': {'include_leaves': False,
  'use_kneepoint': True,
  'use_std': False,
  'std_factor': 1.0},
 'ddist_flattening_config': {'include_leaves': False,
  'use_kneepoint': True,
  'use_std': False,
  'std_factor': 1.0},
 'linkage_args': {'method': 'ward', 'optimal_ordering': False},
 'llm_config': {'type': 'llm_pipeline',
  'id': 1,
  'name': '0001_llm_pipeline',
  'dir_name': 'llm_pipelines',
  'lab_name': 'injection_lab',
  'rel_path': 'injection_lab/llm_pipelines/0001_llm_pipeline',
  'name_or_path': 'meta-llama/Meta-Llama-3.1-8B-Instruct',
  'pipeline_args': {'return_full_text': False,
   'clean_up_tokenization_spaces': True},
  'bnb_config': {'load_in_4bit': True,
   'bnb_4bit_use_double_quant': True,
   'bnb_4bit_quant_type': 'nf4'},
  'inference_tokenizer_config': {'padding_side': 'left',


In [43]:
reimport()

In [47]:
dataset_df=dataset_df[['kidx','tidx','node_id','hpath','prompt_messages','gold_message']].copy()
dataset_df['split']='train'

dataset=BaseDataset(df=dataset_df, parent_ref=6)
dataset_config=dict(
    remarks='with toc, english system prompt, german query prompt',
    tax_config=taxomizer.get_config(),
)
dataset.update_config(dataset_config)
dataset.get_config()
dataset.spawn()

2025-02-07 09:06:00,982 - __init__.py - beesup_llm - __init__ - INFO - 0007_DATASET initialised
2025-02-07 09:06:00,997 - __init__.py - beesup_llm - __init__ - INFO - 0006_DATASET initialised
2025-02-07 09:06:01,329 - __init__.py - beesup_llm - spawn_config - INFO - 0007_DATASET config spawned at injection_lab/datasets/0007_dataset
2025-02-07 09:06:01,330 - __init__.py - root - spawn - INFO - 0007_DATASET spawned at /home/fboehning/fboehning/injection_lab/datasets/0007_dataset


In [48]:
BaseDataset.get_overview(keypaths=['n_rows','remarks','tax_config','split'])


,id,name,remarks,n_rows,tax_config
0,1,0001_dataset,"basic knowledge_df from wildbienen_db, no embe...",NaN,NaN
1,2,0002_dataset,"knowledge dataset with embeddings in subject, ...",NaN,NaN
2,3,0003_dataset,"knowledge dataset with embeddings in subject, ...",NaN,NaN
3,4,0004_dataset,"knowledge dataset with embeddings in subject, ...",NaN,NaN
4,5,0005_dataset,"knowledge dataset with embeddings in subject, ...",NaN,NaN
5,6,0006_dataset,"knowledge dataset with embeddings in subject, ...",NaN,NaN
6,7,0007_dataset,"with toc, english system prompt, german query ...",3412.0,"{'type': 'taxomizer', 'id': 2, 'name': '0002_t..."


# Create Questions

In [347]:
dataset=BaseDataset(7)
dataset_df=dataset.df

knowledge_df=BaseDataset(dataset.parent_config).df

# add embedding combinations
add_cols=['s','p','o','source_name','attr_type','n_units','n_words']
emb_cols=[f"{c}_emb" for c in ['s','p','o','so','po','spo']]
dataset_df=dataset_df.merge(knowledge_df[add_cols+emb_cols], left_on='kidx', right_index=True, how='left')
dataset_df


2025-02-07 14:07:52,064 - __init__.py - beesup_llm - __init__ - INFO - 0007_DATASET initialised
2025-02-07 14:07:52,090 - __init__.py - beesup_llm - __init__ - INFO - 0006_DATASET initialised


,kidx,tidx,node_id,hpath,prompt_messages,gold_message,split,s,p,o,source_name,attr_type,n_units,n_words,s_emb,p_emb,o_emb,so_emb,po_emb,spo_emb
0,1916,1692,6812,"[Wildbienen, Umgangssprachliche Bezeichnungen ...","[{'role': 'system', 'content': 'You are a know...","[{'role': 'assistant', 'content': '""Als Nestpa...",train,"Hylaeus communis (Nylander, 1852)",Parasiten,"""Als Nestparasit wurden Schmalbauchwespen der ...",wikipedia,Text,1,22,"[0.0074964184, -0.006532411, 0.010707644, 0.00...","[0.0067741987, -0.00579156, 0.012198386, 0.006...","[0.006624135, -0.016266031, 0.0042751855, 0.02...","[0.0175043, -0.012922153, -0.0012822307, 0.013...","[0.010434584, -0.023192989, -0.00057288643, 0....","[0.011627321, -0.015591463, -0.001989269, 0.01..."
1,527,458,6812,"[Wildbienen, Umgangssprachliche Bezeichnungen ...","[{'role': 'system', 'content': 'You are a know...","[{'role': 'assistant', 'content': 'Von der Art...",train,"Andrena nigroaenea (Kirby, 1802)",Systematik,Von der Art wurden mehrere Unterarten beschrie...,wikipedia,Text,1,27,"[-0.003798669, -0.025593871, 0.009554575, -0.0...","[0.008728638, -0.0020928406, 0.026165051, -0.0...","[0.003310302, -0.018220898, 0.018383589, -0.00...","[-0.0030643565, -0.026073407, 0.01752103, -0.0...","[0.013902873, -0.018704087, 0.012390775, -0.00...","[-0.00032512893, -0.028085468, 0.017131323, -0..."
2,1070,939,6812,"[Wildbienen, Umgangssprachliche Bezeichnungen ...","[{'role': 'system', 'content': 'You are a know...","[{'role': 'assistant', 'content': 'Die Deichhu...",train,"Bombus distinguendus (Morawitz, 1869)",Kurzbeschreibung,Die Deichhummel (Bombus distinguendus) ist ein...,wikipedia,Text,1,10,"[0.0035069948, -0.030443307, 0.00828194, 0.002...","[0.005835669, 0.00046010816, 0.013336124, 0.00...","[0.0011134875, 0.001320186, 0.0057929945, 0.00...","[0.00798141, -0.002544037, 0.010852625, 0.0044...","[0.0048630238, 0.0067080576, 0.00670563, 0.007...","[0.0026471736, -0.0005944702, 0.012585272, 0.0..."
3,3620,3182,6812,"[Wildbienen, Umgangssprachliche Bezeichnungen ...","[{'role': 'system', 'content': 'You are a know...","[{'role': 'assistant', 'content': 'Sphecodes e...",train,"Sphecodes ephippius (Linnaeus, 1767)",Kurzbeschreibung,Sphecodes ephippius ist eine parasitische Bien...,wikipedia,Text,1,20,"[-0.004589162, -0.027405059, -0.0027844561, 0....","[0.005835669, 0.00046010816, 0.013336124, 0.00...","[-0.010704446, 0.006089516, 0.0037873902, 0.00...","[-0.010260817, 0.003773774, 0.007520293, -0.00...","[-0.010285014, 0.010988846, 0.0028065783, 0.00...","[-0.01181556, 0.0044817245, 0.0042099343, 0.00..."
4,2189,1926,6812,"[Wildbienen, Umgangssprachliche Bezeichnungen ...","[{'role': 'system', 'content': 'You are a know...","[{'role': 'assistant', 'content': 'Die Glocken...",train,"Lasioglossum costulatum (Kriechbaumer, 1873)",Kurzbeschreibung,Die Glockenblumen-Schmalbiene (Lasioglossum co...,wikipedia,Text,1,12,"[0.012257211, -0.014003468, 0.012141728, 0.003...","[0.005835669, 0.00046010816, 0.013336124, 0.00...","[0.0023955784, -0.0008903179, 0.013634946, 0.0...","[0.0080582155, 0.0026082508, 0.017733106, -0.0...","[0.0026733035, 0.0025412277, 0.010954422, 0.00...","[0.007582612, 0.005349722, 0.015507144, -3.243..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3407,3661,3219,3412,"[Wildbienen, Umgangssprachliche Bezeichnungen ...","[{'role': 'system', 'content': 'You are a know...","[{'role': 'assistant', 'content': 'Sphecodes e...",train,"Sphecodes hyalinatus (Hagens, 1882)",Lebensweise,Sphecodes ephippus ist wie alle Arten der Gatt...,wikipedia,Text,1,50,"[0.0031964243, -0.014885427, -0.0057354365, 0....","[0.015797619, 0.0071725207, 0.023838716, -0.00...","[-0.02218734, -0.007309703, 0.021173999, 0.007...","[-0.016588943, -0.008555522, 0.013912918, 0.00...","[-0.017442727, -0.0017777351, 0.023208305, 0.0...","[-0.013539648, -0.0041589825, 0.014827495, 0.0..."
3408,218,190,3412,"[Wildbienen, Umgangssprachliche Bezeichnungen ...","[{'role': 's

In [370]:
from sklearn.metrics.pairwise import cosine_similarity

def fetch_similar_samples(sample, samples_df, emb_col='spo_emb'):
    if not emb_col.endswith('_emb'): emb_col=f"{emb_col}_emb"

    sample_embs=np.vstack(samples_df[emb_col].values)
    sample_emb=sample[emb_col]

    match_matrix=cosine_similarity(sample_emb.reshape(1,-1),sample_embs)[0]

    similar_df=samples_df[['kidx','tidx','s','p','o']].copy()
    similar_df['score']=match_matrix
    

    similar_df.sort_values(by='score',ascending=False,inplace=True)

    similar_df=similar_df[similar_df.kidx!=sample.kidx].copy()
    similar_df.reset_index(drop=True,inplace=True)
    return similar_df

def add_sample(sample, path):
    samples_df=pd.read_pickle(path)
    samples_df=pd.concat([samples_df, pd.DataFrame([sample])], ignore_index=True)
    samples_df.to_pickle(path)
    return samples_df

## MCQ Evaluator

### generate multiple choice questions

In [102]:
def get_mcq_generation_prompt(sample,samples_df, emb_col=None):

    CHUNK_TEMPLATE="{s}\t{p}\t{o}"
    get_mcq_generation_prompt.add_kidc=[]
    
    add_chunks=False
    if sample.attr_type=='Attribute': add_chunks=True

    if emb_col is None: emb_col=random.choice(['s','p','o','so','po','spo'])
    get_mcq_generation_prompt.emb_col=emb_col
    
    prompt=""
    prompt+="Create a multiple-choice question that tests the understanding of the following knowledge chunk. "
    prompt+="Ensure the question is clear, concise, and relevant to the key knowledge in the chunk. "
    prompt+="Provide four answer options, with one correct answer and three plausible but incorrect wrong choices."
    if add_chunks:
        prompt+=" I also will provide some additional knowledge chunks that you can use to make up the wrong choices. "
        prompt+="But the main focus lies on the knowledge chunk initially provided."
    
    prompt+="\n\n"
    prompt+="""
Return the output in the following format as a JSON string:
{
    "question": "Here goes the generated question in German",
    "right_choice": "the correct answer",
    "wrong_choices": ["first wrong answer", "second wrong answer", "third wrong answer"] # List of three wrong answer options
}
""".strip()
    
    prompt+="\n\n"

    if add_chunks: prompt+="### MAIN KNOWLEDGE-CHUNK:\n\n"
    else: prompt+="### KNOWLEDGE-CHUNK:\n\n"
    prompt+=CHUNK_TEMPLATE.format(**sample)+'\n\n'

    if add_chunks:
        similar_df=fetch_similar_samples(sample, samples_df, emb_col=emb_col)

        prompt+="### ADDITIONAL KNOWLEDGE-CHUNKS:\n\n"
        chars, max_chars = 0, 600
        for i,similar_sample in similar_df.iterrows():
            line=CHUNK_TEMPLATE.format(**similar_sample)+'\n\n'
            prompt+=line
            chars+=len(line)
            get_mcq_generation_prompt.add_kidc.append(similar_sample.kidx)
            if chars>max_chars: break
            if i>4: break
    
    prompt+="### QUESTION:\n"

    return prompt


In [105]:
new_samples_df=dataset_df.sample(100, random_state=None).copy()
new_samples_df.reset_index(drop=True, inplace=True)
new_samples_df

,kidx,tidx,node_id,hpath,prompt_messages,gold_message,split,s,p,o,source_name,attr_type,n_units,n_words,s_emb,p_emb,o_emb,so_emb,po_emb,spo_emb
0,635,551,6798,"[Wildbienen, Umgangssprachliche Bezeichnungen ...","[{'role': 'system', 'content': 'You are a know...","[{'role': 'assistant', 'content': 'Die Weibche...",train,"Andrena proxima (Kirby, 1802)",Merkmale,Die Weibchen dieser Sandbiene sind ca. 9 bis 1...,wikipedia,Text,1,98,"[0.0008072834, -0.028953291, 0.016739747, -0.0...","[-0.0061423453, -0.0048425547, 0.019819293, -0...","[0.004521345, 0.0060171573, 0.009871953, -0.00...","[0.008377177, -0.010645693, 0.014183677, -0.01...","[0.0048229573, 0.0028101604, 0.00591885, -0.00...","[0.009204846, -0.011883186, 0.014200246, -0.01..."
1,2543,2227,6807,"[Wildbienen, Nist- und Bauverhalten von Wildbi...","[{'role': 'system', 'content': 'You are a know...","[{'role': 'assistant', 'content': 'Totholz'}]",train,"Lithurgus chrysurus (Fonscolombe, 1834)",Requisiten,Totholz,wildbienen-kataster,Attribute,1,1,"[-0.008092702, -0.010585132, 0.012372567, 0.00...","[0.009611603, 0.002773139, 0.029174019, -0.006...","[-0.0038502081, -0.014879571, 0.025984177, 0.0...","[-0.0149837, -0.013633011, 0.014211607, 0.0012...","[0.008261229, -0.0013367896, 0.031230103, -0.0...","[0.005153158, -0.0057438803, 0.027318576, -0.0..."
2,2886,2533,6797,"[Wildbienen, Umgangssprachliche Bezeichnungen ...","[{'role': 'system', 'content': 'You are a know...","[{'role': 'assistant', 'content': 'Die Art ist...",train,"Nomada castellana (Dusmet y Alonso, 1913)",Vorkommen und Lebensweise,"Die Art ist in Süd-, Mittel- und Osteuropa ver...",wikipedia,Text,1,33,"[-6.084886e-05, -0.012234403, 0.014304865, 0.0...","[0.024019513, 0.0069985026, 0.027007056, -0.00...","[0.0064658187, -0.013199007, 0.02406804, 0.004...","[0.005557789, -0.012217172, 0.027850483, -0.00...","[0.010790672, -0.0103467675, 0.02491543, 0.003...","[0.008645264, -0.0130505245, 0.031027328, 0.00..."
3,3795,3331,6810,"[Wildbienen, Umgangssprachliche Bezeichnungen ...","[{'role': 'system', 'content': 'You are a know...","[{'role': 'assistant', 'content': 'Hoplitis ac...",train,"Stelis ornatula (Klug, 1807)",Wirtsbienen,"Hoplitis acuticornis (Dufour & Perris, 1840); ...",wildbienen-kataster,Attribute,7,31,"[-0.00017535561, -0.02494646, 0.007870729, 0.0...","[0.0021889533, 0.003912768, 0.027358979, -0.01...","[-0.013402254, -0.012226291, -0.006685878, 0.0...","[-0.0074112094, -0.022139786, -0.0013497253, 0...","[-0.008743792, -0.011472839, -0.00366245, -0.0...","[-0.0050220257, -0.015230891, 9.633621e-05, -0..."
4,2644,2318,6807,"[Wildbienen, Nist- und Bauverhalten von Wildbi...","[{'role': 'system', 'content': 'You are a know...","[{'role': 'assistant', 'content': 'Blattstücke'}]",train,"Megachile lagopoda (Linnaeus, 1761)",Baumaterial,Blattstücke,wildbienen-kataster,Attribute,1,1,"[-0.000731122, -0.016388932, 0.007082363, 3.07...","[-0.0052415705, -0.005533913, 0.022423591, -0....","[0.014902174, -0.0060134586, 0.021639897, -0.0...","[0.015076236, -0.00888917, 0.016492717, -0.002...","[0.009664617, -0.0025492227, 0.016610626, -0.0...","[0.014208033, -0.009797516, 0.019821424, -0.00..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,606,526,6806,"[Wildbienen, Nist- und Bauverhalten von Wildbi...","[{'role': 'system', 'content': 'You are a know...","[{'role': 'assistant', 'content': 'endogäisch-...",train,"Andrena polita (Smith, 1847)",Nistweise,endogäisch-selbstgegrabene Hohlräume,wildbienen-kataster,Attribute,1,2,"[0.0033432895, -0.025133671, 0.0035302297, 0.0...","[0.011682943, -0.005001527, 0.036846403, 0.003...","[-0.0002908062, 0.009349991, 0.03383516, 0.007...","[0.00056549674, -0.011211174, 0.018195521, 0.0...","[-0.0020187253, 0.0049945214, 0.038034555, 0.0...","[-0.0030578903, -0.015467134, 0.021294516, 0.0..."
96,3113,2732,6793,"[Wildbienen, Nist- und Bauverhalten von Wildbi...","[{'role': 'system', 'content': 'You are a know...","[{'role': 'assistant', '

In [89]:
import openai
openai.api_key = os.getenv('OPENAI_API_KEY')
client = openai.OpenAI()

logging.getLogger("openai").setLevel(logging.WARNING)
logging.getLogger("httpx").setLevel(logging.WARNING)

In [346]:
print(get_mcq_generation_prompt(sample, dataset_df))

Create a multiple-choice question that tests the understanding of the following knowledge chunk. Ensure the question is clear, concise, and relevant to the key knowledge in the chunk. Provide four answer options, with one correct answer and three plausible but incorrect wrong choices.

Return the output in the following format as a JSON string:
{
    "question": "Here goes the generated question in German",
    "right_choice": "the correct answer",
    "wrong_choices": ["first wrong answer", "second wrong answer", "third wrong answer"] # List of three wrong answer options
}

### KNOWLEDGE-CHUNK:

Andrena proxima (Kirby, 1802)	Merkmale	Die Weibchen dieser Sandbiene sind ca. 9 bis 11 mm lang, die Männchen ca. 8 bis 10 mm. Die Weibchen haben am Hinterleib auf den Tergiten 2 bis 4 schräg nach außen gerichtete Haarbinden, die in der Mitte unterbrochen sind. Die Endfranse ist gelbbraun, die Schienenbürste weiß. Der Kopf ist sehr breit. Die Seitenfelder des Propodeums (Körbchen) sind bedeckt 

In [106]:
for i,sample in new_samples_df.iterrows():

    prompt=get_mcq_generation_prompt(sample, dataset_df)

    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[dict(role='user',content=prompt)],
        ).choices[0].message.content
    
    mcq_sample=dict(
        kidx=sample.kidx,
        tidx=sample.tidx,
        add_kidc=get_mcq_generation_prompt.add_kidc,
        emb_col=get_mcq_generation_prompt.emb_col,
        prompt=prompt,
        completion=completion,
        timestamp=pd.to_datetime(get_timestamp(), format='%Y-%m-%d_%H-%M-%S')
    )

    mcq_samples_df=add_sample(mcq_sample,'mcq_samples_df.pkl')
    print(f"{i+1}/{len(new_samples_df)}"+25*" ", end="\r")
    break

### create MCQ Dataset

In [151]:
#PREPARE WILDBEE MCQ QUESTIONS

def get_mcq_dict(mcq_sample):
    completion=mcq_sample.completion
    completion=re.sub(r'```json\s*([\s\S]*?)\s*```', r'\1', completion.strip())

    try:
        completion_dict=json.loads(completion)
    except Exception as e:
        return dict(question=str(e)) 
    
    choices=[completion_dict['right_choice']]+completion_dict['wrong_choices']
    random.shuffle(choices)
    
    mcq_dict=dict(
        question=completion_dict['question'],
        choices=choices,
        answer=choices.index(completion_dict['right_choice'])
    )

    return mcq_dict

mcq_samples_df=pd.read_pickle('mcq_samples_df.pkl')

evaluator_df=mcq_samples_df.copy()
evaluator_df=evaluator_df[
    (evaluator_df.timestamp > '2025-02-05')
    ]

evaluator_df.reset_index(names='sidx', inplace=True)

# convert to standard mcq format
evaluator_df['mcq_dict']=evaluator_df.apply(get_mcq_dict, axis=1)
evaluator_df = evaluator_df.join(pd.json_normalize(evaluator_df['mcq_dict']))

# display parsing errors
invalid_df=evaluator_df[evaluator_df.answer.isna()].copy()
if not invalid_df.empty:
    logging.warning(f"Found {len(invalid_df)} invalid completion(s)")
    #display(invalid_df)
evaluator_df.drop(index=invalid_df.index, inplace=True)
evaluator_df.reset_index(drop=True, inplace=True)
evaluator_df.drop(columns=['mcq_dict'], inplace=True)

int_cols=['sidx','kidx','tidx','answer']
evaluator_df[int_cols]=evaluator_df[int_cols].astype(int)

use_cols=['sidx','kidx','tidx','add_kidc','add_tidc','emb_col','question','choices','answer']
evaluator_df=evaluator_df[use_cols].copy()

evaluator_df

2025-02-07 10:50:24,609 - 1435031688.py - root - <module> - WARNING - Found 1 invalid completion(s)


,sidx,kidx,tidx,add_kidc,add_tidc,emb_col,question,choices,answer
0,10,1420,1258,NaN,"[1302, 1301, 1300]",s,Welche Bienenarten sind bekannte Wirtsbienen v...,"[Megachile apicalis Spinola, 1808; Megachile l...",0
1,11,991,867,NaN,[],so,Woran kann man die Königin der Westlichen Honi...,[An ihrer Größe und dem verlängerten Hinterlei...,0
2,12,646,560,NaN,"[461, 310, 2160]",po,Welche Kuckucksbiene ist bei Andrena rhenana (...,"[Nomada femoralis Morawitz, 1869, Nomada rufic...",2
3,13,1172,1034,NaN,[],p,In welchem Lebensraum ist die Grauweiße Hummel...,[In den Küstenregionen Südosteuropas unterhalb...,2
4,14,3366,2956,NaN,[],spo,Welches Merkmal unterscheidet das Weibchen der...,"[Das Weibchen ist zwischen 12 und 14 mm lang.,...",1
...,...,...,...,...,...,...,...,...,...
100,111,2103,1854,NaN,"[46, 3398, 1879]",p,Welche Nistweise hat Hylaeus styriacus (Förste...,"[endogäisch-selbstgegrabene Hohlräume, markhal...",2
101,112,3118,2736,NaN,[864],p,Zu welcher Familie gehört Nomada mutica?,"[Apidae, Felidae, Canidae, Hominidae]",0
102,113,2808,2465,NaN,[],s,In welchem geografischen Gebiet ist die Zahntr...,"[Nur in Südengland, Ausschließlich in Schweden...",2
103,114,59,52,NaN,[872],p,Wie lautet der umgangssprachliche Name für die...,"[Honigbiene, Ostbiene, Westliche Zangensandbie...",2


In [176]:
# PREPARE MMLU MCQ QUESTIONS

mmlu_dataset=datasets.load_dataset("cais/mmlu", "all")
mmlu_samples_df=mmlu_dataset['test'].to_pandas()
evaluator_df=mmlu_samples_df.sample(100, random_state=1).copy()
evaluator_df.reset_index(names='mmluidx', inplace=True)
evaluator_df

,mmluidx,question,subject,choices,answer
0,272,The so-called dark energy is a model to explai...,astronomy,"[the radiation of black holes., the mass distr...",2
1,4300,The positive square root of 200 is what percen...,high_school_mathematics,"[29, 14, 27, 11]",0
2,9465,How is iron transported in the circulation fro...,nutrition,"[As simple Fe2+ in the serum, Bound to albumin...",3
3,1060,Which of the following is not a sentence that ...,college_computer_science,"[xyz, xy, xxzy, xxxxy]",0
4,588,What is hypochlorhydria?,clinical_knowledge,"[Increased amount of gastric acid., Gastric ul...",2
...,...,...,...,...,...
95,2090,Find the number that makes the statement true:...,elementary_mathematics,"[32, 3.2, 3,200, 320]",0
96,10188,The Nazca geoglyphs extend over miles and repr...,prehistory,[entrances to subterranean tombs of the elite....,2
97,8965,For which of these two scenarios does the main...,moral_scenarios,"[Wrong, Wrong, Wrong, Not wrong, Not wrong, Wr...",0
98,6148,"With respect to marital satisfaction, most lon...",human_aging,[Show high satisfaction through all the years ...,2


In [153]:
reimport()

In [177]:
evaluator=MCQEvaluator(
    df=evaluator_df,
    llm_ref=llm_pipe,
    remarks="mmlu questions from cais/mmlu testset, 100 rows sampled with random_state=1"
    )

evaluator.get_config()

2025-02-07 11:14:28,600 - __init__.py - beesup_llm - __init__ - INFO - 0004_LLM_EVALUATOR initialised
2025-02-07 11:14:28,616 - __init__.py - beesup_llm - __init__ - INFO - 0001_LLM_PIPELINE initialised


{'type': 'llm_evaluator',
 'id': 4,
 'name': '0004_llm_evaluator',
 'dir_name': 'llm_evaluators',
 'lab_name': 'injection_lab',
 'rel_path': 'injection_lab/llm_evaluators/0004_llm_evaluator',
 'subtype': 'mcq',
 'remarks': 'mmlu questions from cais/mmlu testset, 100 rows sampled with random_state=1',
 'n_rows': 100,
 'llm_config': {'type': 'llm_pipeline',
  'id': 1,
  'name': '0001_llm_pipeline',
  'dir_name': 'llm_pipelines',
  'lab_name': 'injection_lab',
  'rel_path': 'injection_lab/llm_pipelines/0001_llm_pipeline',
  'name_or_path': 'meta-llama/Meta-Llama-3.1-8B-Instruct',
  'pipeline_args': {'return_full_text': False,
   'clean_up_tokenization_spaces': True},
  'bnb_config': {'load_in_4bit': True,
   'bnb_4bit_use_double_quant': True,
   'bnb_4bit_quant_type': 'nf4'},
  'inference_tokenizer_config': {'padding_side': 'left',
   'padding': 'longest',
   'add_special_tokens': True,
   'max_length': 8192,
   'pad_token': '<|begin_of_text|>',
   'pad_token_id': 128000},
  'training_tok

In [178]:
evaluator.spawn()


    

2025-02-07 11:14:35,981 - __init__.py - beesup_llm - spawn_config - INFO - 0004_LLM_EVALUATOR config spawned at injection_lab/llm_evaluators/0004_llm_evaluator
2025-02-07 11:14:35,982 - __init__.py - root - spawn - INFO - 0004_LLM_EVALUATOR spawned at /home/fboehning/fboehning/injection_lab/llm_evaluators/0004_llm_evaluator


In [168]:
reimport()

In [179]:
evaluator=BaseEvaluator.from_ref(4, llm_ref=llm_pipe)
evaluator.get_config()

2025-02-07 11:15:02,621 - __init__.py - beesup_llm - __init__ - INFO - 0004_LLM_EVALUATOR initialised
2025-02-07 11:15:02,636 - __init__.py - beesup_llm - __init__ - INFO - 0001_LLM_PIPELINE initialised


{'type': 'llm_evaluator',
 'id': 4,
 'name': '0004_llm_evaluator',
 'dir_name': 'llm_evaluators',
 'lab_name': 'injection_lab',
 'rel_path': 'injection_lab/llm_evaluators/0004_llm_evaluator',
 'subtype': 'mcq',
 'remarks': 'mmlu questions from cais/mmlu testset, 100 rows sampled with random_state=1',
 'n_rows': 100,
 'llm_config': {'type': 'llm_evaluator',
  'id': 1,
  'name': '0001_llm_pipeline',
  'dir_name': 'llm_pipelines',
  'lab_name': 'injection_lab',
  'rel_path': 'injection_lab/llm_pipelines/0001_llm_pipeline',
  'name_or_path': 'meta-llama/Meta-Llama-3.1-8B-Instruct',
  'pipeline_args': {'return_full_text': False,
   'clean_up_tokenization_spaces': True},
  'bnb_config': {'load_in_4bit': True,
   'bnb_4bit_use_double_quant': True,
   'bnb_4bit_quant_type': 'nf4'},
  'inference_tokenizer_config': {'padding_side': 'left',
   'padding': 'longest',
   'add_special_tokens': True,
   'max_length': 8192,
   'pad_token': '<|begin_of_text|>',
   'pad_token_id': 128000},
  'training_to

In [180]:
eval_df=evaluator()
eval_df=eval_df.join(pd.json_normalize(eval_df['eval_dict']))
print(eval_df.tp.mean())
eval_df

,mmluidx,question,subject,choices,answer,prompt_messages,pred_completion,eval_dict,pred_choice,gold_choice,tp
0,272,The so-called dark energy is a model to explai...,astronomy,"[the radiation of black holes., the mass distr...",2,"[{'role': 'user', 'content': 'You are provided...",C,"{'pred_choice': 'c', 'gold_choice': 'c', 'tp': 1}",c,c,1
1,4300,The positive square root of 200 is what percen...,high_school_mathematics,"[29, 14, 27, 11]",0,"[{'role': 'user', 'content': 'You are provided...",C,"{'pred_choice': 'c', 'gold_choice': 'a', 'tp': 0}",c,a,0
2,9465,How is iron transported in the circulation fro...,nutrition,"[As simple Fe2+ in the serum, Bound to albumin...",3,"[{'role': 'user', 'content': 'You are provided...",D,"{'pred_choice': 'd', 'gold_choice': 'd', 'tp': 1}",d,d,1
3,1060,Which of the following is not a sentence that ...,college_computer_science,"[xyz, xy, xxzy, xxxxy]",0,"[{'role': 'user', 'content': 'You are provided...",A,"{'pred_choice': 'a', 'gold_choice': 'a', 'tp': 1}",a,a,1
4,588,What is hypochlorhydria?,clinical_knowledge,"[Increased amount of gastric acid., Gastric ul...",2,"[{'role': 'user', 'content': 'You are provided...",C,"{'pred_choice': 'c', 'gold_choice': 'c', 'tp': 1}",c,c,1
...,...,...,...,...,...,...,...,...,...,...,...
95,2090,Find the number that makes the statement true:...,elementary_mathematics,"[32, 3.2, 3,200, 320]",0,"[{'role': 'user', 'content': 'You are provided...",D,"{'pred_choice': 'd', 'gold_choice': 'a', 'tp': 0}",d,a,0
96,10188,The Nazca geoglyphs extend over miles and repr...,prehistory,[entrances to subterranean tombs of the elite....,2,"[{'role': 'user', 'content': 'You are provided...",C,"{'pred_choice': 'c', 'gold_choice': 'c', 'tp': 1}",c,c,1
97,8965,For which of these two scenarios does the main...,moral_scenarios,"[Wrong, Wrong, Wrong, Not wrong, Not wrong, Wr...",0,"[{'role': 'user', 'content': 'You are provided...",C),"{'pred_choice': 'c', 'gold_choice': 'a', 'tp': 0}",c,a,0
98,6148,"With respect to marital satisfaction, most lon...",human_aging,[Show high satisfaction through all the years ...,2,"[{'role': 'user', 'content': 'You are provided...",D,"{'pred_choice': 'd', 'gold_choice': 'c', 'tp': 0}",d,c,0


## QDQ - Query Driven Questions

- Attribute-Based Understanding
- test the ability to connect and contextualize information from disparate chunks into a unified understanding.
- only cover knowlege with attr_type=='Attribute'
- Assesses categorization skills by identifying entities that fall into predefined groups based on their attributes.
- the answers cannot be derived from a single chunk but involve synthesizing information from several chunks
- "SQL-like" answer


In [246]:
# get attrs_df
attrs_df=dataset_df[dataset_df['attr_type']=='Attribute'].copy()
attrs_df=attrs_df[['s','p','o']]
attrs_df.rename(
    columns={
        's':'Wissenschaftlicher Name',
        'p':'attribute',
        'o':'value'
        },inplace=True)

attrs_df=attrs_df.pivot(index='Wissenschaftlicher Name', columns='attribute', values='value').reset_index()
attrs_df.columns.name=None
attrs_df = attrs_df.reset_index(drop=True)
attrs_df.fillna('', inplace=True)
attrs_df

,Wissenschaftlicher Name,Baumaterial,Familie,Kuckucksbienen,Kuckuckswespen,Nistweise,Pollensammelverhalten,Requisiten,Sozialverhalten,Umgangssprachliche Bezeichnungen,Wirtsbienen
0,"Aglaoapis tridentata (Nylander, 1848)",,Megachilidae,,,,,,parasitisch,Dunkle Zweizahnbiene; Zweizahnbiene,"Hoplitis anthocopoides (Schenck, 1853); Hoplit..."
1,"Amegilla quadrifasciata (de Villers, 1789)",,Apidae,"Thyreus histrionicus (Illiger, 1806)",,endogäisch-selbstgegrabene Hohlräume,polylektisch,,,Weiße Bindenpelzbiene,
2,"Ammobates punctatus (Fabricius, 1804)",,Apidae,,,,,,parasitisch,Große Sandgängerbiene; Sandgänger-Biene,"Anthophora bimaculata (Panzer, 1798)"
3,"Ammobatoides abdominalis (Eversmann, 1852)",,,,,,,,,Steppenglanzbiene,
4,"Andrena afzeliella (Kirby, 1802)",,Andrenidae,,,endogäisch-selbstgegrabene Hohlräume,,,,Kleine Kleesandbiene; Kleine Kleesandbiene,
...,...,...,...,...,...,...,...,...,...,...,...
506,"Thyreus orbatus (Lepeletier, 1841)",,Apidae,,,,,,parasitisch,Fleckenbiene; Schwarzgesichtige Fleckenbiene,"Anthophora borealis Morawitz, 1864; Anthophora..."
507,"Trachusa byssina (Panzer, 1798)","Blattstücke, Harz",Megachilidae,"Coelioxys conica (Linnaeus, 1758)",,endogäisch-selbstgegrabene Hohlräume,oligolektisch,,,Große Harzbiene,
508,"Xylocopa iris (Christ, 1791)",Mark,Apidae,,,hypergäisch-selbstgegrabene Hohlräume,polylektisch,Pflanzenstängel,,Kleine Holzbiene,
509,"Xylocopa valga (Gerstäcker, 1872)",Holzgenagsel,,,Polochrum repandum Spinola 1806,hypergäisch-selbstgegrabene Hohlräume,,Totholz,,Schwarzfühler-Holzbiene; Südliche Holzbiene; Ö...,


In [334]:
# Query Driven Questions
QDQ_SAMPLES=[
dict(
    conditions=(
        (attrs_df['Umgangssprachliche Bezeichnungen'].str.contains('Natternkopf'))
    ),
    target_attr='Wissenschaftlicher Name',
    question='Welche Wildbienen können als Natternkopf-Bienen bezeichnet werden?'
),
dict(
    conditions=(
        (attrs_df.Familie=='Megachilidae')
        &(attrs_df.Pollensammelverhalten.str.contains('polylektisch'))
        &(attrs_df['Baumaterial'].str.contains('Blattstücke'))
    ),
    target_attr='Wissenschaftlicher Name',
    question='Welche Wildbienen der Familie Megachilidae sammelen Pollen von vielen verschiedenen Pflanzenarten und bauen ihre Nester aus Blattstücken?'
),
dict(
    conditions=(
        (attrs_df.Kuckucksbienen.str.contains('Nomada panzeri'))  
    ),
    target_attr='Familie',
    question='Für die Wildbienen welcher Familien ist Nomada panzeri eine Kuckucksbiene?'
),
dict(
    conditions=(
        (attrs_df['Umgangssprachliche Bezeichnungen'].str.contains('Zwergsandbiene')) 
    ),
    target_attr='Wissenschaftlicher Name',
    question='Welche Wildbienen werden umgangssprachlich als Zwergsandbienen bezeichnet?'
),
dict(
    conditions=(
        (attrs_df.Familie.str.contains('Halictidae'))
        & (attrs_df['Nistweise'].str.contains('endogäisch'))
        & (attrs_df['Pollensammelverhalten'].str.contains('oligolektisch'))
    ),
    target_attr='Wissenschaftlicher Name',
    question='Welche Wildbienen der Familie Halictidae bauen ihre Nester in der Erde und sammeln Pollen von wenigen Pflanzenarten?'
),
dict(
    conditions=(
        (attrs_df['Nistweise'].str.contains('hypergäisch'))
        & (attrs_df['Baumaterial'].str.contains('Harz'))
    ),
    target_attr='Wissenschaftlicher Name',
    question='Welche Wildbienen bauen ihre Nester oberirdisch und verwenden Harz als Baumaterial?'
),
dict(
    conditions=(
        (attrs_df.Familie.str.contains('Halictidae'))
        & (attrs_df['Umgangssprachliche Bezeichnungen'].str.contains('Furchenbiene'))
        & (attrs_df.Pollensammelverhalten.str.contains('polylektisch')) 
    ),
    target_attr='Wissenschaftlicher Name',
    question='Welche Wildbienen der Familie Halictidae werden als Furchenbienen bezeichnet und sammeln Pollen von vielen verschiedenen Pflanzenarten?'
),
dict(
    conditions=(
        (attrs_df['Wissenschaftlicher Name'].str.contains('Morawitz'))
        & (attrs_df['Umgangssprachliche Bezeichnungen'].str.contains('Wespenbiene'))  
    ),
    target_attr='Wissenschaftlicher Name',
    question='Welche Wespenbienen wurden durch Morawitz beschrieben?'
),
dict(
    conditions=(
        (attrs_df.Familie.str.contains('Halictidae'))
        & (attrs_df['Wirtsbienen'].str.contains('Andrena'))       
    ),
    target_attr='Wissenschaftlicher Name',
    question='Welche Wildbienen der Familie Halictidae parasitieren die Gattung Andrena?'
),
dict(
    conditions=(
        (attrs_df['Sozialverhalten'].str.contains('komunal'))  
    ),
    target_attr='Wissenschaftlicher Name',
    question='Welche Wildbienen haben ein komunales Sozialverhalten?'
),
dict(
    conditions=(
        (attrs_df['Umgangssprachliche Bezeichnungen'].str.contains('Filzbiene'))  
    ),
    target_attr='Wissenschaftlicher Name',
    question='Welche Wildbienen werden als Filzbienen bezeichnet?'
),
dict(
    conditions=(
        (attrs_df['Kuckuckswespen'].str.contains('Sapyga'))
        & (attrs_df.Pollensammelverhalten.str.contains('polylektisch')) 
    ),
    target_attr='Wissenschaftlicher Name',
    question='Welche Wildbienen sammeln Pollen von vielen verschiedenen Pflanzenarten und haben Kuckuckswespen der Gattung Sapyga?'
),
dict(
    conditions=(
        (attrs_df['Kuckuckswespen'].str.contains('Sapyga'))
        & (attrs_df.Pollensammelverhalten.str.contains('oligolektisch')) 
    ),
    target_attr='Wissenschaftlicher Name',
    question='Welche Wildbienen sammeln Pollen von wenigen Pflanzenarten und haben Kuckuckswespen der Gattung Sapyga?'
),
dict(
    conditions=(
        (attrs_df['Baumaterial'].str.contains('Lehm'))
    ),
    target_attr='Wissenschaftlicher Name',
    question='Welche Wildbienen verwenden unter anderem Lehm als Baumaterial?'
),
dict(
    conditions=(
        (attrs_df['Nistweise'].str.contains('Insektenfraßgänge'))
        &(attrs_df['Baumaterial'].str.contains('Blattstücke'))
    ),
    target_attr='Wissenschaftlicher Name',
    question='Welche Wildbienen bauen ihre Nester in Insektenfraßgängen und verwenden Blattstücke als Baumaterial?'
),
dict(
    conditions=(
        (attrs_df['Nistweise'].str.contains('Mauerfugen'))
        &(attrs_df.Pollensammelverhalten.str.contains('polylektisch'))
    ),
    target_attr='Wissenschaftlicher Name',
    question='Welche Wildbienen sammeln Pollen von vielen verschiedenen Pflanzenarten und bauen ihre Nester in Mauerfugen?'
),
dict(
    conditions=(
        (attrs_df.Kuckucksbienen.str.contains('Aglaoapis'))
    ),
    target_attr='Wissenschaftlicher Name',
    question='Für welche Wildbienen sind Bienen der Gattung Aglaoapis mögliche Kuckucksbienen?'
),
dict(
    conditions=(
        (attrs_df.Familie.str.contains('Colletidae'))
        &(attrs_df['Requisiten'].str.contains('Sand'))
    ),
    target_attr='Wissenschaftlicher Name',
    question='Welche Wildbienen der Familie Colletidae verwenden Sand als Requisiten?'
),
dict(
    conditions=(
        (attrs_df['Requisiten'].str.contains('Schnecken'))
        &(attrs_df.Kuckucksbienen.str.contains('Stelis odontopyga'))

    ),
    target_attr='Wissenschaftlicher Name',
    question='Welche Wildbienen bauen ihre Nester in Schneckenhäusern und haben Stelis odontopyga als Kuckucksbiene?'
),
dict(
    conditions=(

        (attrs_df['Umgangssprachliche Bezeichnungen'].str.contains('Maskenbiene'))
        &(attrs_df.Pollensammelverhalten.str.contains('breit-oligolektisch'))
    ),
    target_attr='Wissenschaftlicher Name',
    question='Welche Wildbienen sind als Maskenbienen bekannt und zeigen ein breit-oligolektisches Pollensammelverhalten?'
),
dict(
    conditions=(
        (attrs_df['Sozialverhalten'].str.contains('solitär'))
        &(attrs_df['Nistweise'].str.contains('vorhanden'))
    ),
    target_attr='Wissenschaftlicher Name',
    question='Welche Wildbienen haben ein solitäres Sozialverhalten und nutzen bereits vorhandene Nistplätze?'
),
dict(
    conditions=(
        (attrs_df['Kuckucksbienen'].str.contains('Sphecodes'))
    ),
    target_attr='Familie',
    question='Für die Wildbienen welcher Familien ist sind individuen der Gattung Sphecodes Kuckucksbienen?'
),
dict(
    conditions=(
        (attrs_df['Sozialverhalten'].str.contains('solitär'))
        &(~attrs_df['Pollensammelverhalten'].str.contains('polylektisch'))
    ),
    target_attr='Wissenschaftlicher Name',
    question='Welche Wildbienen leben solitär und zeigen kein polyektisches Pollensammelverhalten?'
),
dict(
    conditions=(
        (attrs_df['Sozialverhalten'].str.contains('solitär'))
        &(~attrs_df['Nistweise'].str.contains('hypergäisch'))
        &(attrs_df['Pollensammelverhalten'].str.contains('starke Präf. Apiaceae'))
    ),
    target_attr='Wissenschaftlicher Name',
    question='Welche Wildbienen leben solitär, bauen keine oberirdischen Nester und zeigen beim Sammeln von Pollen eine starke Vorliebe für Apiaceae?'
),
dict(
    conditions=(
        (attrs_df['Sozialverhalten'].str.contains('solitär'))
        &(attrs_df['Requisiten'].str.contains('Steilwände'))
    ),
    target_attr='Wissenschaftlicher Name',
    question='Welche Wildbienen leben solitär und bauen Nester an Steilwänden?'
),
dict(
    conditions=(
        (attrs_df['Wissenschaftlicher Name'].str.contains('Lasioglossum'))
        &(attrs_df['Sozialverhalten'].str.contains('solitär'))
        &(~attrs_df['Kuckucksbienen'].str.contains('Sphecodes'))
    ),
    target_attr='Wissenschaftlicher Name',
    question='Welche Wildbienen haben keine Kuckucksbienen der Gattung Sphecodes, leben solitär und gehören selbst der Gattung Lasioglossum an?'
),
dict(
    conditions=(
        (attrs_df['Wissenschaftlicher Name'].str.contains('Lasioglossum'))
        &(attrs_df['Sozialverhalten'].str.contains('solitär'))
        &(~attrs_df['Kuckucksbienen'].str.contains('Nomada'))
    ),
    target_attr='Wissenschaftlicher Name',
    question='Welche Wildbienen haben keine Kuckucksbienen der Gattung Nomada, leben solitär und gehören selbst der Gattung Lasioglossum an?'
),
dict(
    conditions=(
        (attrs_df['Sozialverhalten'].str.contains('sozial'))
        &(attrs_df['Kuckucksbienen'].str.contains('Sphecodes'))
    ),
    target_attr='Wissenschaftlicher Name',
    question='Welche Wildbienen leben sozial und sind Wirtsbienen der Gattung Sphecodes?'
),
dict(
    conditions=(
        (attrs_df['Sozialverhalten'].str.contains('sozial'))
        &(attrs_df['Nistweise'].str.contains('hypergäisch'))
        &(attrs_df['Kuckucksbienen']=='')
    ),
    target_attr='Wissenschaftlicher Name',
    question='Welche Wildbienen leben sozial in überirdischen Nester und werden nicht durch Kuckucksbienen parasitiert?'
),
# dict(
#     conditions=(
#         #(attrs_df['Wissenschaftlicher Name'].str.contains('Lasioglossum'))
#         (attrs_df['Sozialverhalten'].str.contains('sozial'))
#         &(attrs_df['Nistweise'].str.contains('hypergäisch'))
#         &(attrs_df['Kuckucksbienen']=='')
#         #&(attrs_df['Pollensammelverhalten'].str.contains('starke Präf. Apiaceae'))
#     ),
#     target_attr='Wissenschaftlicher Name',
#     question='Welche Wildbienen leben sozial in überirdischen Nester und werden nicht durch Kuckucksbienen parasitiert?'
# ),

]

print(len(QDQ_SAMPLES))
all_conditions=[str(s['conditions'].to_list()) for s in QDQ_SAMPLES]
if len(set(all_conditions))!=len(all_conditions):
    print("Duplicate conditions found!")

attrs_df[QDQ_SAMPLES[-1]['conditions']]

29


,Wissenschaftlicher Name,Baumaterial,Familie,Kuckucksbienen,Kuckuckswespen,Nistweise,Pollensammelverhalten,Requisiten,Sozialverhalten,Umgangssprachliche Bezeichnungen,Wirtsbienen
139,"Bombus confusus (Schenck, 1861)",,Apidae,,,endogäisch-vorhandene Hohlräume; hypergäisch-K...,polylektisch,,sozial,Samthummel,
142,"Bombus distinguendus (Morawitz, 1869)",,Apidae,,,endogäisch-vorhandene Hohlräume; hypergäisch-K...,polylektisch,,sozial,Deichhummel,
154,"Bombus muscorum (Linnaeus, 1758)",,Apidae,,,"hypergäisch-Krautschicht, vorhand. Hohlräume",polylektisch,,sozial,Mooshummel,
169,"Bombus veteranus (Fabricius, 1793)",,Apidae,,,endogäisch-vorhandene Hohlräume; hypergäisch-K...,polylektisch,,sozial,Sandhummel,


In [267]:
evaluator_df=pd.read_pickle('evaluation_pipelines/0002_evaluation_pipeline/df.pkl')
evaluator_df

,question,target_attr,gold_items
0,Welche Wildbienenarten können als Natternkopf-...,scientific_name,"[Hoplitis anthocopoides (Schenck, 1853), Hopli..."
1,Welche Bienenarten der Megachilidae sammelen P...,scientific_name,"[Megachile leachella (Curtis, 1828), Megachile..."
2,Für welche Wildbienenarten ist die Art Nomada ...,scientific_name,"[Andrena varians (Kirby, 1802), Andrena fulva ..."
3,Welche unterschiedlichen Arten der Zwergsandbi...,scientific_name,"[Andrena semilaevis (Pérez, 1903), Andrena ant..."
4,Welche Wildbienenarten der Familie Halictidae ...,scientific_name,"[Dufourea minuta (Lepeletier, 1841), Rhophitoi..."
5,Welche Wildbienenarten bauen ihre Nester oberi...,scientific_name,"[Anthidiellum strigatum (Panzer, 1804), Megach..."
6,Welche Wildbienenarten der Familie Halictidae ...,scientific_name,"[Halictus quadricinctus (Fabricius, 1776), Hal..."
7,Welche Wespenbienen wurden durch Morawitz besc...,scientific_name,"[Nomada mutabilis (Morawitz, 1870), Nomada sim..."
8,Welche Wildbienenarten der Familie Halictidae ...,scientific_name,"[Sphecodes monilicornis (Kirby, 1802), Sphecod..."
9,Welche Wildbienenarten haben ein komunales Soz...,scientific_name,"[Andrena carantonica (Pérez, 1902), Andrena bu..."


In [277]:
qdq_sample=QDQ_SAMPLES[7]

attrs_df[qdq_sample['conditions']]['Pollensammelverhalten'].to_list()


['', '', '', '', '', '', '']

In [337]:
evaluator_data=[]

for qdq_sample in QDQ_SAMPLES:

    if qdq_sample['target_attr'] not in ['Wissenschaftlicher Name']: continue

    gold_items=attrs_df[qdq_sample['conditions']][qdq_sample['target_attr']].to_list()
    gold_items=[item for item in gold_items if bool(item)]
    gold_items=list(set(gold_items))


    evaluator_data.append(
        dict(
            question=qdq_sample['question'],
            target_attr=qdq_sample['target_attr'],
            gold_items=gold_items
        )
    )

evaluator_df=pd.DataFrame(evaluator_data)
evaluator_df

,question,target_attr,gold_items
0,Welche Wildbienen können als Natternkopf-Biene...,Wissenschaftlicher Name,"[Hoplitis adunca (Panzer, 1798), Hoplitis anth..."
1,Welche Wildbienen der Familie Megachilidae sam...,Wissenschaftlicher Name,"[Megachile nigriventris (Schenck, 1870), Megac..."
2,Welche Wildbienen werden umgangssprachlich als...,Wissenschaftlicher Name,"[Andrena pauxilla (E.Stöckhert, 1935), Andrena..."
3,Welche Wildbienen der Familie Halictidae bauen...,Wissenschaftlicher Name,"[Rophites algirus (Pérez, 1895), Rophites quin..."
4,Welche Wildbienen bauen ihre Nester oberirdisc...,Wissenschaftlicher Name,"[Anthidiellum strigatum (Panzer, 1804), Megach..."
5,Welche Wildbienen der Familie Halictidae werde...,Wissenschaftlicher Name,"[Halictus langobardicus (Blüthgen, 1944), Hali..."
6,Welche Wespenbienen wurden durch Morawitz besc...,Wissenschaftlicher Name,"[Nomada mutica (Morawitz, 1872), Nomada emargi..."
7,Welche Wildbienen der Familie Halictidae paras...,Wissenschaftlicher Name,"[Sphecodes pellucidus (Smith, 1845), Sphecodes..."
8,Welche Wildbienen haben ein komunales Sozialve...,Wissenschaftlicher Name,"[Andrena bucephala (Stephens, 1846), Andrena c..."
9,Welche Wildbienen werden als Filzbienen bezeic...,Wissenschaftlicher Name,"[Epeolus alpinus (Friese, 1893), Epeolus varie..."


In [336]:
reimport()

In [338]:
evaluator=QDQEvaluator(
    df=evaluator_df,
    llm_ref=llm_pipe,
    remarks="qdq questions only for 'Wissenschaftlicher Name' target_attr"
)

evaluator.get_config()

2025-02-07 13:45:13,937 - __init__.py - beesup_llm - __init__ - INFO - 0005_LLM_EVALUATOR initialised
2025-02-07 13:45:13,955 - __init__.py - beesup_llm - __init__ - INFO - 0001_LLM_PIPELINE initialised


{'type': 'llm_evaluator',
 'id': 5,
 'name': '0005_llm_evaluator',
 'dir_name': 'llm_evaluators',
 'lab_name': 'injection_lab',
 'rel_path': 'injection_lab/llm_evaluators/0005_llm_evaluator',
 'subtype': 'qdq',
 'remarks': "qdq questions only for 'Wissenschaftlicher Name' target_attr",
 'n_rows': 27,
 'llm_config': {'type': 'llm_pipeline',
  'id': 1,
  'name': '0001_llm_pipeline',
  'dir_name': 'llm_pipelines',
  'lab_name': 'injection_lab',
  'rel_path': 'injection_lab/llm_pipelines/0001_llm_pipeline',
  'name_or_path': 'meta-llama/Meta-Llama-3.1-8B-Instruct',
  'pipeline_args': {'return_full_text': False,
   'clean_up_tokenization_spaces': True},
  'bnb_config': {'load_in_4bit': True,
   'bnb_4bit_use_double_quant': True,
   'bnb_4bit_quant_type': 'nf4'},
  'inference_tokenizer_config': {'padding_side': 'left',
   'padding': 'longest',
   'add_special_tokens': True,
   'max_length': 8192,
   'pad_token': '<|begin_of_text|>',
   'pad_token_id': 128000},
  'training_tokenizer_config': 

In [339]:
evaluator.spawn()

2025-02-07 13:45:40,222 - __init__.py - beesup_llm - spawn_config - INFO - 0005_LLM_EVALUATOR config spawned at injection_lab/llm_evaluators/0005_llm_evaluator
2025-02-07 13:45:40,223 - __init__.py - root - spawn - INFO - 0005_LLM_EVALUATOR spawned at /home/fboehning/fboehning/injection_lab/llm_evaluators/0005_llm_evaluator


In [340]:
# 25 items = 6 min

eval_df=evaluator()
eval_df

2025-02-07 13:46:45,289 - __init__.py - beesup_llm - get_pred_df - INFO - max_new_tokens: 452


,question,target_attr,gold_items,prompt_messages,pred_completion,eval_dict
0,Welche Wildbienen der Familie Megachilidae sam...,Wissenschaftlicher Name,"[Megachile nigriventris (Schenck, 1870), Megac...","[{'role': 'user', 'content': 'You are provided...","Die Wildbienen der Familie Megachilidae, die P...",NaN
1,Welche Wildbienen werden umgangssprachlich als...,Wissenschaftlicher Name,"[Andrena pauxilla (E.Stöckhert, 1935), Andrena...","[{'role': 'user', 'content': 'You are provided...",Welche Wildbienen werden umgangssprachlich als...,NaN
2,Welche Wildbienen der Familie Halictidae bauen...,Wissenschaftlicher Name,"[Rophites algirus (Pérez, 1895), Rophites quin...","[{'role': 'user', 'content': 'You are provided...","Lasioglossum calcaris (Schulz, 1903); Lasioglo...",NaN
3,Welche Wildbienen bauen ihre Nester oberirdisc...,Wissenschaftlicher Name,"[Anthidiellum strigatum (Panzer, 1804), Megach...","[{'role': 'user', 'content': 'You are provided...",Welche Wildbienen bauen ihre Nester oberirdisc...,NaN
4,Welche Wildbienen der Familie Halictidae werde...,Wissenschaftlicher Name,"[Halictus langobardicus (Blüthgen, 1944), Hali...","[{'role': 'user', 'content': 'You are provided...",Die Furchenbienen der Familie Halictidae sind:...,NaN
5,Welche Wespenbienen wurden durch Morawitz besc...,Wissenschaftlicher Name,"[Nomada mutica (Morawitz, 1872), Nomada emargi...","[{'role': 'user', 'content': 'You are provided...",Welche Wespenbienen wurden durch Morawitz besc...,NaN
6,Welche Wildbienen der Familie Halictidae paras...,Wissenschaftlicher Name,"[Sphecodes pellucidus (Smith, 1845), Sphecodes...","[{'role': 'user', 'content': 'You are provided...","Lasioglossum andrenae (Schletter, 1882); Lasio...",NaN
7,Welche Wildbienen haben ein komunales Sozialve...,Wissenschaftlicher Name,"[Andrena bucephala (Stephens, 1846), Andrena c...","[{'role': 'user', 'content': 'You are provided...","Apis mellifera (Linnaeus, 1758); Bombus prator...",NaN
8,Welche Wildbienen werden als Filzbienen bezeic...,Wissenschaftlicher Name,"[Epeolus alpinus (Friese, 1893), Epeolus varie...","[{'role': 'user', 'content': 'You are provided...",Welche Wildbienen werden als Filzbienen bezeic...,NaN
9,Welche Wildbienen sammeln Pollen von vielen ve...,Wissenschaftlicher Name,"[Hoplitis claviventris (Thomson, 1872), Osmia ...","[{'role': 'user', 'content': 'You are provided...","Colletes hedemannii (Schletter, 1889); Collete...",NaN


## FFQ - Free Form Questions
- General Understanding/ General Abilities because an Answertext must be formulated

### generate samples

In [380]:
def get_ffq_generation_prompt(sample, samples_df, emb_col=None):
    CHUNK_TEMPLATE = "{s}\t{p}\t{o}"
    get_ffq_generation_prompt.add_kidc = []
    
    if emb_col is None:
        emb_col = random.choice(['s', 'p', 'o', 'so', 'po', 'spo'])
    get_ffq_generation_prompt.emb_col = emb_col
    
    prompt = ""
    prompt += "Create a free-form question that requires an open-ended response based strictly on the provided knowledge chunks. "
    prompt += "Ensure the question is clear, focuses on a single topic, and does not introduce external knowledge. "
    prompt += "The question should be answerable solely using the information contained in the given chunks, especially the main chunk. "
    prompt += "If additional knowledge chunks are provided, use them only to enrich the question while maintaining the main knowledge chunk as the primary focus. "
    prompt += "The generated answer should be concise, ideally one or two sentences long. "
    
    prompt += "\n\n"
    prompt += """
Return the output in the following format as a JSON string:
{
    "question": "Here goes the generated open-ended question in German",
    "answer": "Here goes the optimal answer in one or two sentences, strictly based on the provided knowledge chunks"
}
""".strip()
    
    prompt += "\n\n"
    prompt += "### MAIN KNOWLEDGE-CHUNK:\n\n"

    prompt += CHUNK_TEMPLATE.format(**sample) + '\n\n'

    if not len(CHUNK_TEMPLATE.format(**sample))>400:

        similar_df = fetch_similar_samples(sample, samples_df, emb_col=emb_col)
        prompt += "\n### ADDITIONAL KNOWLEDGE-CHUNKS:\n\n"
        chars, max_chars = 0, 600
        for i, similar_sample in similar_df.iterrows():
            line = CHUNK_TEMPLATE.format(**similar_sample) + '\n\n'
            prompt += line
            chars += len(line)
            get_ffq_generation_prompt.add_kidc.append(similar_sample.kidx)
            if chars > max_chars:
                break
            if i > 2:
                break
    
    prompt += "### QUESTION:\n"
    
    return prompt

In [379]:
len(sample.o)

1045

In [388]:
new_samples_df=dataset_df[dataset_df.attr_type=='Text'].sample(100, random_state=1).copy()
sample=new_samples_df.sample(1).iloc[0]

prompt=get_ffq_generation_prompt(sample, dataset_df)
print(prompt)

#sample = dataset_df.sample(1).iloc[0]

Create a free-form question that requires an open-ended response based strictly on the provided knowledge chunks. Ensure the question is clear, focuses on a single topic, and does not introduce external knowledge. The question should be answerable solely using the information contained in the given chunks, especially the main chunk. If additional knowledge chunks are provided, use them only to enrich the question while maintaining the main knowledge chunk as the primary focus. The generated answer should be concise, ideally one or two sentences long. 

Return the output in the following format as a JSON string:
{
    "question": "Here goes the generated open-ended question in German",
    "answer": "Here goes the optimal answer in one or two sentences, strictly based on the provided knowledge chunks"
}

### MAIN KNOWLEDGE-CHUNK:

Nomada trapeziformis (Schmiedeknecht, 1882)	Vorkommen und Lebensweise	Die Art ist in Süddeutschland, Österreich und Tschechien verbreitet. Die Tiere fliegen i

In [389]:
ffq_sample=dict(
    kidx=sample.kidx,
    tidx=sample.tidx,
    add_kidc=get_ffq_generation_prompt.add_kidc,
    emb_col=get_ffq_generation_prompt.emb_col,
    prompt=prompt,
    completion=None,
    timestamp=pd.to_datetime(get_timestamp(), format='%Y-%m-%d_%H-%M-%S')
)

ffq_sample['completion']="""
{
    "question": "Welche Wirtsarten parasitiert Nomada trapeziformis vermutlich?",
    "answer": "Nomada trapeziformis parasitiert vermutlich Andrena barbareae und Andrena limata."
}
""".strip()

ffq_samples_df=add_sample(ffq_sample,'ffq_samples_df.pkl')
ffq_samples_df.tail(5)


,kidx,tidx,add_kidc,emb_col,prompt,completion,timestamp
0,3028,2657,[2968],spo,Create a free-form question that requires an o...,"{\n ""question"": ""Woran kann man die Männche...",2025-02-07 15:36:18
1,2022,1783,"[2024, 2021]",s,Create a free-form question that requires an o...,"{\n ""question"": ""Wie unterscheiden sich die...",2025-02-07 15:40:14
2,222,194,[],so,Create a free-form question that requires an o...,"{\n ""question"": ""Warum werden die Arten der...",2025-02-07 15:41:17
3,2943,2582,"[1924, 814]",p,Create a free-form question that requires an o...,"{\n ""question"": ""Zu welcher Familie gehört ...",2025-02-07 15:42:06
4,3284,2882,"[3298, 3194, 2987]",po,Create a free-form question that requires an o...,"{\n ""question"": ""Welche Wirtsarten parasiti...",2025-02-07 15:43:21


In [ ]:
import openai
openai.api_key = os.getenv('OPENAI_API_KEY')
client = openai.OpenAI()

logging.getLogger("openai").setLevel(logging.WARNING)
logging.getLogger("httpx").setLevel(logging.WARNING)

In [ ]:
for i,sample in new_samples_df.iterrows():

    prompt=get_ffq_generation_prompt(sample, dataset_df)

    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[dict(role='user',content=prompt)],
        ).choices[0].message.content
    
    ffq_sample=dict(
        kidx=sample.kidx,
        tidx=sample.tidx,
        add_kidc=get_ffq_generation_prompt.add_kidc,
        emb_col=get_ffq_generation_prompt.emb_col,
        prompt=prompt,
        completion=completion,
        timestamp=pd.to_datetime(get_timestamp(), format='%Y-%m-%d_%H-%M-%S')
    )

    ffq_samples_df=add_sample(ffq_sample,'ffq_samples_df.pkl')
    print(f"{i+1}/{len(new_samples_df)}"+25*" ", end="\r")
    break

### create evaluator

In [391]:
#PREPARE WILDBEE MCQ QUESTIONS

def get_ffq_dict(ffq_sample):
    completion=ffq_sample.completion
    completion=re.sub(r'```json\s*([\s\S]*?)\s*```', r'\1', completion.strip())

    try:
        completion_dict=json.loads(completion)
    except Exception as e:
        return dict(question=str(e)) 
  
    mcq_dict=dict(
        question=completion_dict['question'],
        answer=completion_dict['answer']
    )

    return mcq_dict

ffq_samples_df=pd.read_pickle('ffq_samples_df.pkl')

evaluator_df=ffq_samples_df.copy()
evaluator_df=evaluator_df[
    (evaluator_df.timestamp > '2025-02-05')
    ]

evaluator_df.reset_index(names='sidx', inplace=True)

# convert to standard mcq format
evaluator_df['ffq_dict']=evaluator_df.apply(get_ffq_dict, axis=1)
evaluator_df = evaluator_df.join(pd.json_normalize(evaluator_df['ffq_dict']))

# display parsing errors
invalid_df=evaluator_df[evaluator_df.answer.isna()].copy()
if not invalid_df.empty:
    logging.warning(f"Found {len(invalid_df)} invalid completion(s)")
    #display(invalid_df)
evaluator_df.drop(index=invalid_df.index, inplace=True)
evaluator_df.reset_index(drop=True, inplace=True)
evaluator_df.drop(columns=['ffq_dict'], inplace=True)

int_cols=['sidx','kidx','tidx']
evaluator_df[int_cols]=evaluator_df[int_cols].astype(int)

use_cols=['sidx','kidx','tidx','add_kidc','emb_col','question','answer']
evaluator_df=evaluator_df[use_cols].copy()

evaluator_df

,sidx,kidx,tidx,add_kidc,emb_col,question,answer
0,0,3028,2657,[2968],spo,Woran kann man die Männchen und Weibchen von N...,Die Männchen von Nomada guttulata sehen den We...
1,1,2022,1783,"[2024, 2021]",s,Wie unterscheiden sich die Männchen und Weibch...,Die Weibchen von Hylaeus nigritus haben eine d...
2,2,222,194,[],so,Warum werden die Arten der Untergattung Cnemid...,Die Arten der Untergattung Cnemidandrena werde...
3,3,2943,2582,"[1924, 814]",p,Zu welcher Familie gehört die Bienenart Nomada...,Nomada femoralis gehört zur Familie der Apidae.
4,4,3284,2882,"[3298, 3194, 2987]",po,Welche Wirtsarten parasitiert Nomada trapezifo...,Nomada trapeziformis parasitiert vermutlich An...


In [392]:
reimport()

In [393]:
evaluator=FFQEvaluator(
    df=evaluator_df,
    llm_ref=llm_pipe,
    remarks="free form questions from gpt-4o"
    )

evaluator.get_config()

2025-02-07 14:51:29,898 - __init__.py - beesup_llm - __init__ - INFO - 0006_LLM_EVALUATOR initialised
2025-02-07 14:51:29,915 - __init__.py - beesup_llm - __init__ - INFO - 0001_LLM_PIPELINE initialised


{'type': 'llm_evaluator',
 'id': 6,
 'name': '0006_llm_evaluator',
 'dir_name': 'llm_evaluators',
 'lab_name': 'injection_lab',
 'rel_path': 'injection_lab/llm_evaluators/0006_llm_evaluator',
 'subtype': 'ffq',
 'remarks': 'free form questions from gpt-4o',
 'n_rows': 5,
 'llm_config': {'type': 'llm_pipeline',
  'id': 1,
  'name': '0001_llm_pipeline',
  'dir_name': 'llm_pipelines',
  'lab_name': 'injection_lab',
  'rel_path': 'injection_lab/llm_pipelines/0001_llm_pipeline',
  'name_or_path': 'meta-llama/Meta-Llama-3.1-8B-Instruct',
  'pipeline_args': {'return_full_text': False,
   'clean_up_tokenization_spaces': True},
  'bnb_config': {'load_in_4bit': True,
   'bnb_4bit_use_double_quant': True,
   'bnb_4bit_quant_type': 'nf4'},
  'inference_tokenizer_config': {'padding_side': 'left',
   'padding': 'longest',
   'add_special_tokens': True,
   'max_length': 8192,
   'pad_token': '<|begin_of_text|>',
   'pad_token_id': 128000},
  'training_tokenizer_config': {'padding_side': 'right',
   '

In [394]:
evaluator.spawn()

2025-02-07 14:51:49,242 - __init__.py - beesup_llm - spawn_config - INFO - 0006_LLM_EVALUATOR config spawned at injection_lab/llm_evaluators/0006_llm_evaluator
2025-02-07 14:51:49,244 - __init__.py - root - spawn - INFO - 0006_LLM_EVALUATOR spawned at /home/fboehning/fboehning/injection_lab/llm_evaluators/0006_llm_evaluator


In [395]:
eval_df=evaluator()
eval_df

,sidx,kidx,tidx,add_kidc,emb_col,question,answer,prompt_messages,pred_completion,eval_dict
0,0,3028,2657,[2968],spo,Woran kann man die Männchen und Weibchen von N...,Die Männchen von Nomada guttulata sehen den We...,"[{'role': 'user', 'content': 'Woran kann man d...",Die Unterscheidung zwischen den Männchen und W...,{'info': 'NotImplemented'}
1,1,2022,1783,"[2024, 2021]",s,Wie unterscheiden sich die Männchen und Weibch...,Die Weibchen von Hylaeus nigritus haben eine d...,"[{'role': 'user', 'content': 'Wie unterscheide...",Die Maskenbiene Hylaeus nigritus ist eine Art ...,{'info': 'NotImplemented'}
2,2,222,194,[],so,Warum werden die Arten der Untergattung Cnemid...,Die Arten der Untergattung Cnemidandrena werde...,"[{'role': 'user', 'content': 'Warum werden die...",Die Arten der Untergattung Cnemidandrena werde...,{'info': 'NotImplemented'}
3,3,2943,2582,"[1924, 814]",p,Zu welcher Familie gehört die Bienenart Nomada...,Nomada femoralis gehört zur Familie der Apidae.,"[{'role': 'user', 'content': 'Zu welcher Famil...",Die Bienenart Nomada femoralis gehört zur Fami...,{'info': 'NotImplemented'}
4,4,3284,2882,"[3298, 3194, 2987]",po,Welche Wirtsarten parasitiert Nomada trapezifo...,Nomada trapeziformis parasitiert vermutlich An...,"[{'role': 'user', 'content': 'Welche Wirtsarte...",Ich konnte keine genauen Informationen über di...,{'info': 'NotImplemented'}


In [396]:
Evaluator.get_overview(keypaths=['n_rows','remarks'])

,id,name,remarks,n_rows
0,1,0001_llm_evaluator,104 wild bee knowledge questions from dataset ...,NaN
1,2,0002_llm_evaluator,"100 general questions from cais/mmlu, sampled ...",NaN
2,3,0003_llm_evaluator,"wildbee mcq questions, mainly generated by gpt...",105.0
3,4,0004_llm_evaluator,"mmlu questions from cais/mmlu testset, 100 row...",100.0
4,5,0005_llm_evaluator,qdq questions only for 'Wissenschaftlicher Nam...,27.0
5,6,0006_llm_evaluator,free form questions from gpt-4o,5.0


# Experiment

In [77]:
trainwrap=LoraTrainerWrap()
trainwrap.get_config()
trainwrap.spawn()

2025-02-03 12:44:42,042 - __init__.py - beesup_llm - __init__ - INFO - 0001_LORA_TRAINER initialised
2025-02-03 12:44:42,053 - __init__.py - beesup_llm - spawn_config - INFO - 0001_LORA_TRAINER config spawned at injection_lab/lora_trainers/0001_lora_trainer


In [ ]:
experiment=InjectionExperiment(
    
)